In [4]:
from __future__ import print_function
import torch
import torchtext
from torchtext.datasets import text_classification
NGRAMS = 3
import os
if not os.path.isdir('data'):
    os.mkdir('data')
train_dataset, test_dataset = text_classification.DATASETS['AG_NEWS'](root='data', ngrams=NGRAMS, vocab=None)
BATCH_SIZE = 16
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

120000lines [00:07, 15359.54lines/s]
120000lines [00:17, 6756.04lines/s]
7600lines [00:01, 7474.85lines/s]


In [5]:
import torch.nn as nn
import torch.nn.functional as F
class TextSentiment(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

In [6]:
VOCAB_SIZE = len(train_dataset.get_vocab())
EMBED_DIM = 32
NUN_CLASS = len(train_dataset.get_labels())
model = TextSentiment(VOCAB_SIZE, EMBED_DIM, NUN_CLASS).to(device)

In [7]:
def generate_batch(batch):
    label = torch.tensor([entry[0] for entry in batch])
    text = [entry[1] for entry in batch]
    offsets = [0] + [len(entry) for entry in text]
    # torch.Tensor.cumsum returns the cumulative sum
    # of elements in the dimension dim.
    # torch.Tensor([1.0, 2.0, 3.0]).cumsum(dim=0)

    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text = torch.cat(text)
    return text, offsets, label

In [8]:
from torch.utils.data import DataLoader

def train_func(sub_train_):

    # Train the model
    train_loss = 0
    train_acc = 0
    data = DataLoader(sub_train_, batch_size=BATCH_SIZE, shuffle=True,
                      collate_fn=generate_batch)
    for i, (text, offsets, cls) in enumerate(data):
        optimizer.zero_grad()
        text, offsets, cls = text.to(device), offsets.to(device), cls.to(device)
        output = model(text, offsets)
        loss = criterion(output, cls)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
        train_acc += (output.argmax(1) == cls).sum().item()

    # Adjust the learning rate
    scheduler.step()

    return train_loss / len(sub_train_), train_acc / len(sub_train_)

def test(data_):
    loss = 0
    acc = 0
    data = DataLoader(data_, batch_size=BATCH_SIZE, collate_fn=generate_batch)
    for text, offsets, cls in data:
        text, offsets, cls = text.to(device), offsets.to(device), cls.to(device)
        with torch.no_grad():
            output = model(text, offsets)
            loss = criterion(output, cls)
            loss += loss.item()
            acc += (output.argmax(1) == cls).sum().item()

    return loss / len(data_), acc / len(data_)

In [9]:
import time
from torch.utils.data.dataset import random_split
N_EPOCHS = 5
min_valid_loss = float('inf')

criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=4.0)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.9)

train_len = int(len(train_dataset) * 0.95)
sub_train_, sub_valid_ = \
    random_split(train_dataset, [train_len, len(train_dataset) - train_len])

for epoch in range(N_EPOCHS):

    start_time = time.time()
    train_loss, train_acc = train_func(sub_train_)
    valid_loss, valid_acc = test(sub_valid_)

    secs = int(time.time() - start_time)
    mins = secs / 60
    secs = secs % 60

    print('Epoch: %d' %(epoch + 1), " | time in %d minutes, %d seconds" %(mins, secs))
    print("\tLoss: %.4f (train)\t|\t Acc: %.1f (train)"%(train_loss, train_acc * 100))
    print("\tLoss: %.4f (val)\t|\t Acc: %.1f (val)"%(valid_loss, valid_acc * 100))

)
torch.Size([2124])
torch.Size([2133])
torch.Size([1815])
torch.Size([1977])
torch.Size([2037])
torch.Size([2295])
torch.Size([1953])
torch.Size([1917])
torch.Size([2103])
torch.Size([2124])
torch.Size([2151])
torch.Size([2124])
torch.Size([1896])
torch.Size([1767])
torch.Size([1998])
torch.Size([1980])
torch.Size([2130])
torch.Size([2007])
torch.Size([2028])
torch.Size([1878])
torch.Size([2187])
torch.Size([2106])
torch.Size([2064])
torch.Size([2085])
torch.Size([2160])
torch.Size([1926])
torch.Size([2025])
torch.Size([1878])
torch.Size([2058])
torch.Size([2049])
torch.Size([1986])
torch.Size([1866])
torch.Size([2112])
torch.Size([1953])
torch.Size([1950])
torch.Size([1875])
torch.Size([1893])
torch.Size([2106])
torch.Size([1989])
torch.Size([1983])
torch.Size([2193])
torch.Size([2001])
torch.Size([1923])
torch.Size([1998])
torch.Size([2190])
torch.Size([2067])
torch.Size([2004])
torch.Size([2094])
torch.Size([1899])
torch.Size([2193])
torch.Size([2220])
torch.Size([2367])
torch.Size

In [10]:
print('Checking the results of test dataset...')
test_loss, test_acc = test(test_dataset)
print("\tLoss: %.4f (test)\t|\t Acc: %.1f (test)"%(test_loss, test_acc * 100))

Checking the results of test dataset...
	Loss: 0.0007 (test)	|	 Acc: 75.1 (test)


In [13]:
import re
from torchtext.data.utils import ngrams_iterator
from torchtext.data.utils import get_tokenizer

ag_news_label = {1 : "World",
                 2 : "Sports",
                 3 : "Business",
                 4 : "Sci/Tec"}

def predict(text, model, vocab, ngrams):
    tokenizer = get_tokenizer("basic_english")
    with torch.no_grad():
        print("token: ", tokenizer(text))
        print("ngram", [token for token in ngrams_iterator(tokenizer(text), ngrams)])
        text = torch.tensor([vocab[token]
                            for token in ngrams_iterator(tokenizer(text), ngrams)])
        print(text)
        
        output = model(text, torch.tensor([0]))
        return output.argmax(1).item() + 1

ex_text_str = "MEMPHIS, Tenn. – Four days ago, Jon Rahm was \
    enduring the season’s worst weather conditions on Sunday at The \
    Open on his way to a closing 75 at Royal Portrush, which \
    considering the wind and the rain was a respectable showing. \
    Thursday’s first round at the WGC-FedEx St. Jude Invitational \
    was another story. With temperatures in the mid-80s and hardly any \
    wind, the Spaniard was 13 strokes better in a flawless round. \
    Thanks to his best putting performance on the PGA Tour, Rahm \
    finished with an 8-under 62 for a three-stroke lead, which \
    was even more impressive considering he’d never played the \
    front nine at TPC Southwind."

vocab = train_dataset.get_vocab()
model = model.to("cpu")

print("This is a %s news" %ag_news_label[predict(ex_text_str, model, vocab, 2)])

token:  ['memphis', ',', 'tenn', '.', '–', 'four', 'days', 'ago', ',', 'jon', 'rahm', 'was', 'enduring', 'the', 'season’s', 'worst', 'weather', 'conditions', 'on', 'sunday', 'at', 'the', 'open', 'on', 'his', 'way', 'to', 'a', 'closing', '75', 'at', 'royal', 'portrush', ',', 'which', 'considering', 'the', 'wind', 'and', 'the', 'rain', 'was', 'a', 'respectable', 'showing', '.', 'thursday’s', 'first', 'round', 'at', 'the', 'wgc-fedex', 'st', '.', 'jude', 'invitational', 'was', 'another', 'story', '.', 'with', 'temperatures', 'in', 'the', 'mid-80s', 'and', 'hardly', 'any', 'wind', ',', 'the', 'spaniard', 'was', '13', 'strokes', 'better', 'in', 'a', 'flawless', 'round', '.', 'thanks', 'to', 'his', 'best', 'putting', 'performance', 'on', 'the', 'pga', 'tour', ',', 'rahm', 'finished', 'with', 'an', '8-under', '62', 'for', 'a', 'three-stroke', 'lead', ',', 'which', 'was', 'even', 'more', 'impressive', 'considering', 'he’d', 'never', 'played', 'the', 'front', 'nine', 'at', 'tpc', 'southwind', '